# Ad Quality Analyzer Training Notebook
This notebook demonstrates loading the clickbait dataset, training a baseline model, and evaluating performance.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
#loading datasets
cb = pd.read_csv('clickbait_data.csv')
cb = cb[['headline','clickbait']].rename(columns={'headline':'text'})
cb['label'] = cb['clickbait'].map({1:'spam', 0:'neutral'})
cb = cb[['text','label']]

off = pd.read_csv('labeled_data.csv')
# In this file: class==2 → neither; class==0 or 1 → offensive
off = off[['tweet','class']].rename(columns={'tweet':'text'})
off['label'] = off['class'].apply(lambda c: 'neutral' if c==2 else 'offensive')
off = off[['text','label']]

#combining both datasets
df = pd.concat([cb, off], ignore_index=True)
print(df['label'].value_counts())

label
offensive    20620
neutral      20164
spam         15999
Name: count, dtype: int64


In [3]:
#vectorizing using Tfidf
vec = TfidfVectorizer(max_features=5_000, ngram_range=(1,2))
X = vec.fit_transform(df['text'])
le = LabelEncoder()
y = le.fit_transform(df['label'])   # maps ['neutral','offensive','spam'] → [0,1,2]


In [4]:
#train/test split (stratify to keep class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

In [5]:
#applying logistic regression
model = LogisticRegression(
    multi_class='multinomial', solver='lbfgs', max_iter=1_000
)
model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [6]:

#evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

     neutral       0.93      0.94      0.94      3025
   offensive       0.96      0.97      0.97      3093
        spam       0.96      0.95      0.96      2400

    accuracy                           0.95      8518
   macro avg       0.95      0.95      0.95      8518
weighted avg       0.95      0.95      0.95      8518

Confusion Matrix:
 [[2847  104   74]
 [  84 2991   18]
 [ 114    6 2280]]


In [9]:
#testing function
def predict_text(text):
    X_ = vec.transform([text])
    probs = model.predict_proba(X_)[0]
    for cls, p in zip(le.classes_, probs):
        print(f"{cls:10s}: {p:.2f}")
    print("→  Predicted:", le.inverse_transform(model.predict(X_))[0])

#try it out yourself :)
predict_text("You won’t believe this hack!!!")
predict_text("Our quarterly report is out—download it now")
predict_text("Shes such a pain in the ass!")

neutral   : 0.03
offensive : 0.04
spam      : 0.93
→  Predicted: spam
neutral   : 0.63
offensive : 0.15
spam      : 0.22
→  Predicted: neutral
neutral   : 0.11
offensive : 0.85
spam      : 0.04
→  Predicted: offensive
